In [1]:
import pandas as pd
from tqdm import tqdm

## 0. Загрузка и обзор данных

In [2]:
data = pd.read_excel("отзывы за лето.xls")
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [3]:
data['Rating'].value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

## 1. Обработка текстов

In [4]:
import nltk
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from string import punctuation

In [5]:
stop = set(stopwords.words('russian'))
# stop -= {'не', 'нельзя', 'нет', 'ни', 'нибудь', 'никогда', 'ничего'}
# punctuation = punctuation.replace('-', '')

morpher = MorphAnalyzer()

Функции для обработки текстов:

In [6]:
def normalize_text(text_list):
    return [morpher.parse(word)[0].normal_form \
            for word in text_list if word not in stop]

def preprocess_text(text):    
    text_list = nltk.word_tokenize(str(text))
    text_list = normalize_text(text_list)
    return text_list

In [7]:
data['Text'] = data['Content'].str.lower()
data['Text'] = data['Text'].str.replace(r'[0-9{}]'.format(punctuation), ' ')
data['Text'] = [preprocess_text(text_list) for text_list in tqdm(data['Text'])]

100%|███████████████████████████████████| 20659/20659 [00:50<00:00, 405.71it/s]


In [8]:
# data['Text'] = data['Text'].str.join(' ')
# data['Text'] = data['Text'].str.replace('не ', 'не')
# data['Text'] = data['Text'].str.split()

Датафрейм с обработанными текстами:

In [9]:
data.head(n=5)

,Rating,Content,Date,Text
0,5,It just works!,2017-08-14,"[it, just, works]"
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14,"[целое, удобноной, приложение, минус, хотеть, ..."
2,5,Отлично все,2017-08-14,[отлично]
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14,"[стать, зависать, работа, антивирус, далёкий, ..."
4,5,"Очень удобно, работает быстро.",2017-08-14,"[очень, удобно, работать, быстро]"


## 2. Обучение LDA

In [10]:
from gensim.models.ldamodel import LdaModel
from gensim import corpora

In [11]:
id2word = corpora.Dictionary(data['Text'])
corpus = [id2word.doc2bow(text_list) for text_list in data['Text'] ]

print(id2word.num_nnz) # cумма количеств каждого из слов в каждом тексте

112063


In [12]:
%%time
lda_model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=5,
    passes=10,
    random_state=42,
    minimum_probability=0,
    update_every=1,
    chunksize=100,
    alpha='auto',
    per_word_topics=True
)

Wall time: 1min 24s


Темы и описывающие их топ-слова:

In [13]:
lda_model.print_topics()

[(0,
  '0.086*"удобный" + 0.049*"быстро" + 0.043*"свой" + 0.033*"банк" + 0.030*"вообще" + 0.028*"платёж" + 0.028*"код" + 0.024*"возможность" + 0.023*"смс" + 0.022*"root"'),
 (1,
  '0.053*"обновление" + 0.035*"телефон" + 0.027*"рута" + 0.025*"антивирус" + 0.022*"раз" + 0.021*"последний" + 0.019*"пароль" + 0.017*"стать" + 0.016*"вход" + 0.016*"сделать"'),
 (2,
  '0.054*"нравиться" + 0.047*"отлично" + 0.043*"просто" + 0.038*"супер" + 0.033*"карта" + 0.031*"перевод" + 0.026*"ошибка" + 0.026*"устраивать" + 0.024*"деньга" + 0.022*"исправить"'),
 (3,
  '0.030*"это" + 0.027*"прошивка" + 0.027*"сбербанк" + 0.027*"мочь" + 0.024*"писать" + 0.014*"функция" + 0.014*"зайти" + 0.013*"счёт" + 0.013*"такой" + 0.012*"время"'),
 (4,
  '0.227*"приложение" + 0.095*"очень" + 0.082*"удобно" + 0.066*"работать" + 0.064*"весь" + 0.035*"спасибо" + 0.027*"пользоваться" + 0.025*"хороший" + 0.025*"отличный" + 0.019*"который"')]

Сложно что-то сказать об адекватности результата.